In [ ]:
import os 


In [9]:
from llama_index.core.settings import Settings
from neo4j_graphrag.embeddings import SentenceTransformerEmbeddings
from neo4j import GraphDatabase
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.retrievers import VectorCypherRetriever
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.llms.openai import OpenAI
from pydantic import BaseModel, HttpUrl,Field
from typing import List, Optional, Any
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.agent.workflow import FunctionAgent
from typing import Literal, Dict
from llama_index.core.agent.workflow import AgentInput, AgentOutput, ToolCall, ToolCallResult, AgentStream
from llama_index.core.workflow import Context


In [3]:
llm_gemini = GoogleGenAI(model="gemini-1.5-pro")
llm_open= OpenAI(model="gpt-4o-mini")

In [4]:
import asyncio
from neo4j import AsyncGraphDatabase

NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="password"
NEO4J_URI="bolt://localhost:7687"

driver = AsyncGraphDatabase.driver(
    NEO4J_URI, 
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

def get_session():
    return driver.session()

async def close_driver():
    """Closes the global Neo4j driver."""
    await driver.close()

## Helper Function 

In [5]:
import math

from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

def get_embedding(text: str) -> list[float]:
    if not text or not text.strip():
        return []

    embedding = embed_model.get_text_embedding(text)

    if not isinstance(embedding, list):
        try:
            embedding = embedding.tolist()
        except Exception as e:
            return []

    if not all(isinstance(x, (float, int)) and math.isfinite(x) for x in embedding):
        return []

    return embedding


## Discovery  Agent  

In [6]:

async def extract_node(node_name: str, node_label: Literal["File", "Folder", "Class", "Method"]) -> Dict[str, str]:
    """Useful to extract exactly one repository entity from the user query."""
    return {"node_label": node_label, "node_name": node_name}


async def search_graph(node_label: Literal["File", "Folder", "Class", "Method"], node_name: str ) -> str :
    """Usefull to search for spacific node in Graph databse"""
    top_k: int = 5
    name_embedding = get_embedding(node_name)  

    cypher = f"""
    CALL db.index.vector.queryNodes('{node_label.lower()}_embedding_name_index', $top_k, $embedding)
    YIELD node, score
    {f"WHERE '{node_label}' IN labels(node)" if node_label else ""}
    RETURN node.name AS name,node.description AS description,node.content AS content, score
    ORDER BY score DESC
    """

    async with get_session() as session:  # Assumes get_session can be used as context manager
        result = await session.run(cypher, {
            "embedding": name_embedding,
            "top_k": top_k
        })
        records = [r async for r in result]

    parts: List[str] = []
    for r in records:
        name = r["name"]
        desc = f": {r['description']}" if r.get("description") else ""
        content = r["content"].rstrip()      # trim any trailing whitespace
        parts.append(
            f"- **{name}**\n"
            f"**Description:** {desc}\n\n"
            f"**Code:**\n```\n{content}\n```\n"
        )

    return "\n".join(parts)


In [7]:

### ✅ ** `DISCOVERY_PROMPT`**
DISCOVERY_PROMPT = """
You are DiscoveryAgent — an autonomous retrieval agent responsible for identifying and searching exactly one entity (File, Folder, Class, or Method) from a user's query and searching a graph database for relevant information.

Your task is **not to answer the user directly**, but to **report your findings back to the PlannerAgent**, which will synthesize the final response.

You have access to the following tools:

1. **extract_node(node_name: str, node_label: Literal["File", "Folder", "Class", "Method"])**
   - Extracts a specific entity from the user's query.

2. **search_graph(node_label: str, node_name: str)**
   - Searches the graph database using the extracted node’s label and name.
   - Returns relevant nodes, descriptions, and code snippets in markdown format.

---

### 🔁 Workflow

1. **Understand the Query:**
   - Identify the most relevant entity (e.g., main.py, UserController, etc.).
   - Determine the entity's type: "File", "Folder", "Class", or "Method".

2. **Extract Entity:**
   - Use extract_node with the selected name and type.

3. **Search the Graph:**
   - Use search_graph with **exact output** from extract_node.

4. **Report Findings and Handoff:**
   - Format your response to the **PlannerAgent** using this structure:[DiscoveryAgent Response]
- **Entity**: `main.py` (Type: File)
- **Reason**: Identified as the main file mentioned in the query.
- **Search Results**:
  - **Node**: main.py
    **Score**: 0.98
    **Description**: Entry point for the backend application.
python
    if __name__ == "__main__":
        app.run()
    Summary: This is the application entry point which launches the server.

   - **After generating this report, hand off execution to the PlannerAgent.**

---

### 🔒 Strict Rules

- **One entity only.**
- **Always use extract_node before search_graph.**
- **Use exact tool outputs.**
- **Never invent facts — use only returned data.**
- **Respond only to the PlannerAgent in the specified format.**
- **Always hand off to the PlannerAgent after reporting your findings.**

You are a specialist focused on retrieval. Report clean, structured results back to the planner so it can assemble the final answer and continue the conversation.
"""

discovery_agent = FunctionAgent(
   name="DiscoveryAgent",
   description="DiscoveryAgent is an autonomous retrieval agent that identifies, extracts, and searches exactly one code entity (File, Folder, Class, or Method) from a query using a graph database. It reports its findings in a specific format *only* to the PlannerAgent and then hands off execution for further synthesis and user response.",
   tools=[extract_node, search_graph],
   llm=llm_gemini,
   system_prompt=DISCOVERY_PROMPT, # Use the updated prompt
   can_handoff_to= ["PlannerAgent"]
)



## Relation Resolver Agent

In [21]:
async def get_depend(filename: str, direction: Literal["out", "in"]) -> List[Dict[str, Any]]:
    """Get full node objects of dependencies related to a given file."""
    if direction == "out":
        cypher = """
        MATCH (f:File {name: $filename})-[:RELATED_TO]->(dep:File)
        RETURN dep AS node
        """
    else:
        cypher = """
        MATCH (f:File {name: $filename})<-[:RELATED_TO]-(dep:File)
        RETURN dep AS node
        """

    async with get_session() as session:
        result = await session.run(cypher, {"filename": filename.strip()})
        records = [r async for r in result]

    cleaned_nodes = []
    for record in records:
        node = record.values()[0]
        if hasattr(node, "items"):
            cleaned = {k: v for k, v in dict(node).items() if not k.startswith("embedding")}
            cleaned_nodes.append(cleaned)

    return cleaned_nodes


In [22]:
async def get_node_relationships_by_label(
    label: Literal["File", "Folder", "Class", "Method"],
    name: str,
    direction: Literal["out", "in", "both"],  
    relationship_type: Literal["CONTAINS", "RELATED_TO"],           
):
    """Fetch relationships of a node with the given label and name.
    Excludes any node properties that start with 'embedding'.
    """
    limit = 25
    rel_filter = f":{relationship_type}" if relationship_type else ""

    if direction == "out":
        cypher = f"""
        MATCH (n:{label} {{name: $name}})-[r{rel_filter}]->(m)
        RETURN type(r) AS rel_type, labels(m) AS target_labels, m AS target_node
        LIMIT $limit
        """
    elif direction == "in":
        cypher = f"""
        MATCH (m)-[r{rel_filter}]->(n:{label} {{name: $name}})
        RETURN type(r) AS rel_type, labels(m) AS target_labels, m AS target_node
        LIMIT $limit
        """
    else:  # both
        cypher = f"""
        MATCH (n:{label} {{name: $name}})
        OPTIONAL MATCH (n)-[r1{rel_filter}]->(m1)
        OPTIONAL MATCH (m2)-[r2{rel_filter}]->(n)
        RETURN 
            type(r1) AS out_rel, labels(m1) AS out_labels, m1 AS out_node,
            type(r2) AS in_rel, labels(m2) AS in_labels, m2 AS in_node
        LIMIT $limit
        """

    async with get_session() as session:
        result = await session.run(cypher, {"name": name, "limit": limit})
        records = [record async for record in result] 

    relationships = []

    for record in records:
        if direction in ("out", "both") and record.get("out_rel") and record.get("out_node"):
            node = {k: v for k, v in dict(record["out_node"]).items() if not k.startswith("embedding")}
            relationships.append({
                "direction": "out",
                "relationship_type": record["out_rel"],
                "target_labels": record["out_labels"],
                "target_node": node,
            })
        if direction in ("in", "both") and record.get("in_rel") and record.get("in_node"):
            node = {k: v for k, v in dict(record["in_node"]).items() if not k.startswith("embedding")}
            relationships.append({
                "direction": "in",
                "relationship_type": record["in_rel"],
                "target_labels": record["in_labels"],
                "target_node": node,
            })
        if direction in ("out", "in") and record.get("rel_type") and record.get("target_node"):
            node = {k: v for k, v in dict(record["target_node"]).items() if not k.startswith("embedding")}
            relationships.append({
                "direction": direction,
                "relationship_type": record["rel_type"],
                "target_labels": record["target_labels"],
                "target_node": node,
            })

    return relationships


In [23]:
async def find_path_between_nodes_by_label(
    start_label: Literal["File", "Folder", "Class", "Method"],
    start_name: str,
    end_label: Literal["File", "Folder", "Class", "Method"],
    end_name: str,
    relationship_filter: Literal["CONTAINS", "RELATED_TO"],
):
    """Finds the shortest path between two nodes via a specific relationship type and label."""
    max_depth = 5

    # Relationship filter must be injected directly into the Cypher query
    cypher = f"""
    MATCH path = shortestPath(
        (start:{start_label} {{name: $start_name}})-[:{relationship_filter}*..{max_depth}]-(end:{end_label} {{name: $end_name}})
    )
    RETURN nodes(path) AS nodes, relationships(path) AS relationships
    """

    async with get_session() as session:
        result = await session.run(cypher, {
            "start_name": start_name,
            "end_name": end_name
        })
        records = [record async for record in result]

    paths = []
    for record in records:
        node_path = [
            {k: v for k, v in dict(n).items() if not k.startswith("embedding")}
            for n in record["nodes"]
        ]
        rel_path = [r.type for r in record["relationships"]]
        paths.append({
            "nodes": node_path,
            "relationships": rel_path
        })

    return paths


In [24]:
async def get_full_path_to_node(
    target_label: Literal["File", "Folder", "Class", "Method"],
    target_name: str
):
    """
    Finds the full hierarchical path (using :CONTAINS relationships)
    from the root node labeled 'Repository' down to the specified target node.
    """
    cypher = f"""
    MATCH (root:Repository)
    MATCH path = (root)-[:CONTAINS*]->(target:{target_label} {{name: $target_name}})
    RETURN [n in nodes(path) | n.name] AS path_names // Return the list of node names in order
    """

    async with get_session() as session:
        result = await session.run(cypher, {"target_name": target_name})
        records = await result.data()

    paths_as_strings = []
    for record in records:
        path_names = record.get("path_names")
        if path_names:
            paths_as_strings.append("/".join(path_names))

    return paths_as_strings

In [25]:
RELATION_PROMPT = """
You are the RelationResolverAgent, a specialized agent within a multi-agent system designed to resolve questions about relationships, dependencies, and structural hierarchies in a codebase represented as a graph database.

Your mission is not to respond directly to the user, but to:
- Interpret a relationship-related subtask delegated to you by the PlannerAgent.
- Execute graph queries using the tools provided.
- Return your synthesized results back to the PlannerAgent for final user-facing synthesis.

You have access to the following tools:
1. `get_depend(filename: str, direction: Literal["out", "in", "both"])`: Finds files that a given file depends on ("out") or files that depend on the given file ("in") using 'RELATED_TO' relationships.

2. `get_node_relationships_by_label(label: Literal["File", "Folder", "Class", "Method"], name: str, direction: Literal["out", "in", "both"], relationship_type: Literal["CONTAINS","RELATED_TO"])`: Gets direct one-hop relationships for the given node.

3. `find_path_between_nodes_by_label(start_label: Literal["File", "Folder", "Class", "Method"], start_name: str, end_label: Literal["File", "Folder", "Class", "Method"], end_name: str, relationship_filter: Literal["CONTAINS","RELATED_TO"])`: Finds shortest paths between two entities.

4. `get_full_path_to_node(target_label: Literal["File", "Folder", "Class", "Method"], target_name: str)`: Finds the full hierarchical path from the Repository root to the given node.

Workflow:
1. **Analyze the user's query** as delegated from the Planner. Understand if it’s about:
   - Dependencies ("depends on", "used by", "imports", etc.)
   - Containment or structure ("inside", "parent folder", "full path", etc.)
   - Connections ("path between", "related to", "calls", etc.)

2. **Identify the involved node(s)** — label and name — from the Planner’s handoff.

3. **Choose the correct tool(s)**:
   - Use `get_full_path_to_node` for "full path" or structural location.
   - Use `find_path_between_nodes_by_label` for paths between two nodes.
   - Use `get_depend` for file-level dependencies.
   - Use `get_node_relationships_by_label` for other local or class-level connections.

   For Class or Method dependency questions:
   - **Step 1:** Use `get_node_relationships_by_label` with direction="out".
   - **Step 2:** Use `get_node_relationships_by_label` with direction="in", relationship_type="CONTAINS" to find the containing File.
   - **Step 3:** If a containing file is found, use `get_depend` on that file (direction="out").

4. **Execute the graph queries** with correct parameters.

5. **Interpret and combine results** clearly:
   - If combining direct and file-level dependencies, label them accordingly.
   - Format paths, relationships, or hierarchies in readable bullet or step form.
   - Avoid duplication and filter irrelevant noise.
   - If no data is found, return a meaningful message (e.g., "No dependencies found for X").

6. **Do NOT reply directly to the user.**
   - Instead, hand off the final synthesized answer using:
     `handoff("PlannerAgent", response="<your_final_answer_here>")`

7. **Do NOT perform further reasoning after handoff.** Wait for PlannerAgent to handle any follow-up.

Example Flows:
- Query: “What does `main.py` depend on?”
   → Get dependents using `get_depend("main.py", direction="out")`
   → Return dependencies via `handoff(PlannerAgent, response=...)`

- Query: “What’s the full path to `main.py`?”
   → Use `get_full_path_to_node(label="File", name="main.py")`
   → Return path string via handoff

- Query: “What’s the relationship between `main.py` and `config` folder?”
   → Use `find_path_between_nodes_by_label(...)`
   → Return path/connection details via handoff

Be factual, concise, and helpful. Your only output should be a well-written string summarizing what was found, then immediately handed off to the PlannerAgent.
"""


# The relre_agent definition remains the same, only the system_prompt is updated
relre_agent = FunctionAgent(
   name="RelationResolverAgent",
   description="Resolves dependencies, relationships, and structural paths between entities in the codebase graph, and hands the results back to the PlannerAgent for final response synthesis.",
   tools=[find_path_between_nodes_by_label, get_node_relationships_by_label, get_depend, get_full_path_to_node],
   llm=llm_gemini, 
   system_prompt=RELATION_PROMPT,
   can_handoff_to=["PlannerAgent"]
)

In [28]:
response = await relre_agent.run("what main.py file depend on ?")

/home/younis/miniconda3/envs/llamaindex/lib/python3.12/site-packages/llama_index/core/workflow/events.py:68: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if k in self.model_fields:


In [32]:
response

/home/younis/miniconda3/envs/llamaindex/lib/python3.12/site-packages/llama_index/core/workflow/events.py:81: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if __name in self.__private_attributes__ or __name in self.model_fields:


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='')]), tool_calls=[ToolCallResult(tool_name='get_depend', tool_kwargs={'filename': 'main.py', 'direction': 'out'}, tool_id='get_depend', tool_output=ToolOutput(content='[{\'summary\': \'This module provides utility functions for handling image files and processing delivery images using OCR. It includes functions for saving images, loading images, and performing OCR on delivery images using a generative AI model. The module also defines asynchronous functions for saving files.\', \'path\': \'delivery-ocr/api/src/service/utils.py\', \'name\': \'utils.py\', \'description\': \'This file defines functions for handling image files, specifically for processing delivery images using OCR (Optical Character Recognition). It includes functions for saving images (both synchronously and asynchronously), loading images, and processing delivery im

## QA Agent

In [15]:
from typing import Literal # Make sure Literal is imported

async def similarity_search( node_label: Literal["File", "Class", "Method"], query: str):
    """
    Searches the code graph for nodes (Files, Classes, or Methods)
    semantically similar to the query using vector embeddings.
    Searches within both description and content fields.
    Returns the top_k most relevant nodes and their scores.
    """
    top_k=5 
    embedding = get_embedding(query) # If get_embedding is sync and blocking, this is a problem


    indexes = [
        f"{node_label.lower()}_embedding_description_index",
        f"{node_label.lower()}_embedding_content_index"
    ]

    combined_results = []

    async with get_session() as session:
        for index in indexes:
            cypher = """
            CALL db.index.vector.queryNodes($index_name, $top_k, $embedding)
            YIELD node, score
            RETURN node.name AS name,
                   node.description AS description,
                   node.content AS content,
                   labels(node) AS labels,
                   score
            ORDER BY score DESC
            """
            # Use the 'session' obtained from the 'async with' block
            result = await session.run(cypher, {
                "index_name": index,
                "embedding": embedding,
                "top_k": top_k # Use the local top_k variable
            })

            # Correctly iterate over the async result cursor
            combined_results.extend([record async for record in result])

    seen = set()
    deduped = []
    # Sort the combined results by score (highest first) before deduplicating and truncating
    for record in sorted(combined_results, key=lambda r: r["score"], reverse=True):
        key = record.get("name") # Use get("name") as it's explicitly returned
        if key and key not in seen:
            seen.add(key)
            deduped.append(record)
        elif not key:
             deduped.append(record) 

    return deduped[:top_k]



RESEARCH_PROMPT="""
You are the ResearcherAgent, an expert in navigating and searching a codebase represented as a graph database.
Your primary function is to find specific code elements (Files, Classes, or Methods) relevant to a user's query using semantic search.

Your ONLY tool is `similarity_search`.

Tool:
`similarity_search(node_label: Literal["File", "Class", "Method"], query: str)`
Description: Performs a vector similarity search against nodes of the specified type (File, Class, or Method) using the provided natural language query. It searches both the semantic description and content embeddings. Returns a list of the most relevant nodes found, ordered by relevance score.

Your Workflow:
Your Workflow:
1.  Analyze the user's query carefully to understand the intent.
2.  **Crucially, classify the user's intent to determine the MOST appropriate `node_label`** for the `similarity_search` tool.
    * If the user asks about a specific file or content *within* a file ("find the file that...", "what file contains...", "show me the code in file X"), choose `node_label="File"`.
    * If the user asks about a class definition, purpose, or how a class is used ("what is class Y", "definition of class Z", "how does Class A work"), choose `node_label="Class"`.
    * If the user asks about a specific method/function, its implementation details, or how to perform an action ("how to call method M", "implementation of function F", "show me method N"), choose `node_label="Method"`.
    * If the query is general, try to infer the most likely target. Make your best guess based on keywords. **You must select exactly one label.**
3.  Formulate a clear and concise `query` string to pass to the `similarity_search` tool. This query should capture the essence of what the user is looking for. You can refine the user's original phrasing slightly for better search results.
4.  Call the `similarity_search` tool with the chosen `node_label` and the formulated `query`.
5.  Process the results returned by the tool.
    * Identify the most promising results based on their `score`.
    * Prepare a structured report of your findings.
    * **For each relevant result, include:**
        * `name`
        * `score`
        * `description` (if it exists)
        * `content` (if it exists - this is the code/longer text)
        * `labels`
    * If no results are returned by the tool, state clearly that no relevant results were found for the query under the chosen label.
6.  **ALWAYS** hand off to the `PlannerAgent` after completing the search and preparing your report. Your role is research; the PlannerAgent is responsible for synthesizing the information and formulating the final user-facing response or planning the next steps.
7.  **Your output MUST be formatted ONLY for the PlannerAgent.** Present the search results clearly in a structured format (e.g., a list of dictionaries or a similar readable structure) followed by the handoff command.

Example Handoff Format (After Tool Call):
ResearcherAgent (Handoff):
Research Complete. Found the following results:
[
  {{ "name": "AuthService.py", "score": 0.85, "description": "Handles user authentication flows.", "content": "import hashlib...", "labels": ["File", "Method"] }},
  {{ "name": "UserRepository.java", "score": 0.70, "description": "Manages user data and persistence.", "content": "public class UserRepository { ... }", "labels": ["Class", "File"] }}
]
Handing off to PlannerAgent to process these results.
"""


research_agent = FunctionAgent(
   name="ResearcherAgent",
   description="Researches the codebase using semantic search based on user queries. Analyzes queries to search relevant Files, Classes, or Methods. It reports its findings, including relevant descriptions and potentially code or content snippets, in a specific format *only* to the PlannerAgent and then hands off execution for further synthesis and user response.",
   tools=[similarity_search],
   llm=llm_gemini,
   system_prompt=RESEARCH_PROMPT,
   can_handoff_to= ["PlannerAgent"] 
)



## Planner Agent 

In [ ]:
PLANNER_PROMPT = """
You are the Planner Agent — the reasoning and coordination core of a multi-agent system that answers user questions about a codebase indexed in a graph database.

Your role:
- Analyze the user’s query and understand its intent.
- Decompose the query into one or more precise, meaningful subtasks.
- Delegate these subtasks to the appropriate specialized agents.
- Verify intermediate results before proceeding.
- Synthesize and deliver a complete, accurate, and well-structured response, including relevant code examples where helpful for explaining code functionality.

Agents you can delegate to:
1. **DiscoveryAgent**
   - Use when the query mentions a specific entity such as a file (e.g., `main.py`), folder (e.g., `backend`), class, or method, and you need to **locate or get basic details/content** about that entity.

2. **RelationResolverAgent**
   - Use when the query asks about the **relationship, connection, dependency, or structural path** involving one or more entities.
   - This includes questions like:
     - “How is X related to Y?”
     - “What does X depend on?”
     - “Which entities depend on Y?”
     - “What is the **full path** to Z?”
     - “What is the structure/hierarchy within folder A?”
   - This agent requires the relevant entity/entities to be known and validated before delegation.

3. **ResearcherAgent** (QA Agent)
   - Use when the query is general, fuzzy, or conceptual, and does **not** name any specific file, folder, class, or method as the primary subject asking about its *relationships*, *content*, or *basic existence*.
   - Use **only** when the query is about a general topic (e.g., “How is logging handled?”) OR when the user asks for a higher-level summary/purpose of an *already found* entity, and Discovery's basic description/content is insufficient.

Reasoning process:
- **Crucial Rule:** First, identify if the query mentions any specific entities (File, Folder, Class, Method).
- **If specific entities are mentioned:**
    - **Always** use the `DiscoveryAgent` to locate and verify **all** mentioned entities first.
    - If any mentioned entity is NOT found by Discovery, stop processing that query part and inform the user which entity was not found.
    - If **all** mentioned entities are found:
        - Re-evaluate the original query intent based on the *found* entities.

        - If the core question is about the **relationship, connection, dependency, or structural path** involving these entities (including asking for a "full path" to one of them), delegate the *specific relationship/path task* to the `RelationResolverAgent`. 
            - ⚠️ **Important**: Do NOT try to infer relationships or paths yourself from Discovery results — that is the RelationResolverAgent’s job.

        - If the core question is about the **general purpose or conceptual role** of one of the found entities, and Discovery's information is insufficient, delegate to the `ResearcherAgent`.

        - If the core question was simply to **find the entity and get its basic details/content**, use the results directly from Discovery.

- **If NO specific entities are mentioned:**
    - Proceed directly to general research/QA using the `ResearcherAgent`.

- ✅ For queries like “What does X depend on?” or “What is the relationship between X and Y?”:
    - First locate X (and Y if applicable) using DiscoveryAgent.
    - Then, **always follow up** with RelationResolverAgent to answer the dependency or relationship part.
    - Never return just the Discovery result for such questions — it’s not enough.

- Never assume entities exist — verify through extraction before delegating tasks that depend on those entities being found.
- Never skip a step or respond with partial results unless an entity was not found.

Communication:
- When acknowledging a query and outlining the plan *to the user*, provide a high-level description of the steps using action verbs (e.g., "Searching for the requested file," "Analyzing its content," "Investigating the relationship," "Determining the full location"). Avoid mentioning specific agent names.
- Report your internal reasoning and decisions clearly in the agent's *internal* thought process logs.
- Synthesize the final response comprehensively, combining information obtained from the relevant agents.

Examples:

1. **Query**: "What is the purpose of `main.py`?"
   - **Plan for user**: Okay, I understand the query. I will search to locate the `main.py` file and then analyze its content and description to determine its purpose.
   - **Internal steps**: Use DiscoveryAgent to find `main.py`. If found, use Discovery's description/content. If deeper understanding is needed, call ResearcherAgent for analysis.

2. **Query**: "How is logging handled?"
   - **Plan for user**: Okay, I understand the query. I will research how logging is handled in the codebase.
   - **Internal steps**: Use ResearcherAgent (QA) since no specific entity is the subject.

3. **Query**: "What’s the relation between `main.py` and the `backend` folder?"
   - **Plan for user**: Okay, I understand the query. I will first search to locate `main.py` and the `backend` folder, and then investigate the relationship between them.
   - **Internal steps**: Use DiscoveryAgent to find `main.py`. Use DiscoveryAgent to find `backend`. If both found, delegate to RelationResolverAgent. If any missing, inform user.

4. **Query**: "What does `database.py` depend on?"
   - **Plan for user**: Okay, I understand the query. I will first search to locate the `database.py` file and then determine its dependencies.
   - **Internal steps**: Use DiscoveryAgent to find `database.py`. If found, use RelationResolverAgent to get its dependencies. Do not return Discovery result alone.

5. **Query**: "What is the full path to `main.py`?"
   - **Plan for user**: Okay, I understand the query. I will first search to locate the `main.py` file and then determine its full path.
   - **Internal steps**: Use DiscoveryAgent to find `main.py`. If found, delegate to RelationResolverAgent. Do NOT try to calculate the path yourself.

Think like a software engineer. Verify, reason, then respond. Focus on *what* you are doing for the user, not *which internal tool* you are using.
"""


# The planner_agent definition remains the same, only the system_prompt is updated
planner_agent = FunctionAgent(
   name="PlannerAgent",
   description="The strategic reasoning core that analyzes user queries, breaks them into subtasks, delegates to specialized agents, and synthesizes the final response.",
   tools=[], 
   llm=llm_gemini, 
   system_prompt=PLANNER_PROMPT,
   can_handoff_to=["ResearcherAgent", "RelationResolverAgent", "DiscoveryAgent"] # Using 'can_handoff_to'
)

## Core Agent

In [33]:
from llama_index.core.agent.workflow import AgentWorkflow

insight_agent=AgentWorkflow(
    agents=[planner_agent,discovery_agent, research_agent, relre_agent],
    root_agent=planner_agent.name,

)

In [36]:
handler = insight_agent.run(
    user_msg=" how main file related to utils file?"
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    elif isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)
    
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )


🤖 Agent: PlannerAgent



Okay, I understand the query. I will first search to locate the `main` file and the `utils` file, and then investigate the relationship between them.

📤 Output: Okay, I understand the query. I will first search to locate the `main` file and the `utils` file, and then investigate the relationship between them.


🛠️  Planning to use tools: ['handoff']

🤖 Agent: DiscoveryAgent

🛠️  Planning to use tools: ['extract_node']
🛠️  Planning to use tools: ['search_graph']
🛠️  Planning to use tools: ['extract_node']
🛠️  Planning to use tools: ['search_graph']
[DiscoveryAgent Response]
- **Entity**: `main.py` (Type: File)
- **Reason**: main file is related to utils file because it imports and uses functions from it.
- **Search Results**:
  - **Node**: main.py
  - **Node**: utils.py
    **Description**: utils.py includes functions for handling image files, specifically for processing delivery images using OCR.  The core functionality revolves around saving uploaded image files, loading them, and the